# Example of Graph Neural Network

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.decomposition import PCA
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger

sys.path.append('../..')
device = "cuda" if torch.cuda.is_available() else "cpu"

## Attention Mechanism

In [3]:
from LightningModules.GNN.Models.agnn import ResAGNN
from LightningModules.GNN.Models.vanilla_agnn import VanillaResAGNN
from LightningModules.GNN.Models.checkpoint_agnn import CheckpointedResAGNN
from LightningModules.GNN.Models.interaction_multistep_gnn import CheckpointedInteractionMultistepGNN

In [4]:
with open("example_gnn.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
# model = VanillaResAGNN(hparams)

In [6]:
model = CheckpointedResAGNN(hparams)

In [7]:
# model = CheckpointedInteractionMultistepGNN(hparams)

### Dataset

In [8]:
%%time
model.setup(stage="fit")

CPU times: user 274 µs, sys: 4.13 ms, total: 4.4 ms
Wall time: 4.19 ms


In [9]:
sample = model.trainset[0]

In [10]:
sample

Data(cell_data=[63740, 11], edge_index=[2, 402137], event_file="/project/projectdirs/m3443/data/ITk-upgrade/processed/full_events_v4/event000000001", hid=[63740], modulewise_true_edges=[2, 56793], nhits=[63740], pid=[63740], primary=[63740], pt=[63740], signal_true_edges=[2, 56502], x=[63740, 3], y=[402137], y_pid=[402137])

In [11]:
sample.y.sum()/sample.signal_true_edges.shape[1]

tensor(0.9361)

In [12]:
sample.y.sum()/sample.edge_index.shape[1]

tensor(0.1315)

In [13]:
edges = sample.edge_index

In [14]:
pid = sample.pid

In [15]:
edges.shape

torch.Size([2, 402137])

In [16]:
(sample.pid[edges[0]] == sample.pid[edges[1]]).sum()/sample.edge_index.shape[1]

tensor(0.2224)

### Memory Test

In [17]:
%%time
model.setup(stage="fit")

CPU times: user 370 µs, sys: 4.23 ms, total: 4.6 ms
Wall time: 4.35 ms


In [18]:
sample = model.trainset[0].to(device)

In [19]:
model = model.to(device)

In [20]:
torch.cuda.reset_peak_memory_stats()
output = model(sample.x.to(device), sample.edge_index.to(device))

In [21]:
print(torch.cuda.max_memory_allocated()/1024**3, "Gb")

4.124407768249512 Gb


### Train GNN

In [22]:
import ninja

In [23]:
from pytorch_lightning.plugins import DeepSpeedPlugin

In [24]:
model.setup(stage="fit")

In [25]:
import torch

In [ ]:
logger = WandbLogger(project="ITk_0.5GeV_GNN", group="Gnn_train_1500")
trainer = Trainer(gpus=1, max_epochs=10, logger=logger, precision=16, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/gnn_example/")
trainer.fit(model)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: exatrkx (use `wandb login --relogin` to force relogin)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | edge_network  | Sequential | 32.7 K
1 | node_network  | Sequential | 21.6 K
2 | input_network | Sequential | 13.2 K
---------------------------------------------
67.6 K    Trainable params
0         Non-trainable params
67.6 K    Total params
0.135     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Validation sanity check:  50%|█████     | 1/2 [00:01<00:01,  1.10s/it]

...


Epoch 3:  97%|█████████▋| 1548/1600 [1:06:28<02:13,  2.58s/it, loss=0.26, v_num=zb3a]

/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



Validating:  48%|████▊     | 48/100 [00:31<00:32,  1.59it/s]

/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



Epoch 4:  72%|███████▏  | 1156/1600 [50:35<19:25,  2.63s/it, loss=0.191, v_num=zb3a] 

In [ ]:
print(torch.cuda.max_memory_allocated()/1024**3, "Gb")

FP16

In [ ]:
print(torch.cuda.max_memory_allocated()/1024**3, "Gb")

## Load Model

In [ ]:
checkpoint_path = "/global/cfs/cdirs/m3443/usr/ryanliu/gnn_example/ITk_0.5GeV_GNN/1ummswz1/checkpoints/epoch=0-step=4.ckpt"
checkpoint = torch.load(checkpoint_path)

model = VanillaResAGNN.load_from_checkpoint(checkpoint_path).to(device)

In [ ]:
model.eval();

In [ ]:
model.hparams["datatype_split"] = [200, 1, 10]
model.setup(stage="fit")

In [ ]:
model = model.to(device)

In [ ]:
output_dir = "/global/cfs/cdirs/m3443/usr/ryanliu/gnn_example/ITk_0.5GeV_GNN/"

In [ ]:
with torch.no_grad():
    for batch in model.train_dataloader():

        print(batch)

        output = model.shared_evaluation(batch.to(device), 0, log=False)

        print(output)
        print(os.path.split(batch.event_file[0])[-1])

        gnn_results = np.vstack([torch.cat([batch.edge_index, batch.edge_index.flip(0)], dim=-1).cpu().numpy(), 
                                 output["score"].cpu().numpy(), 
                                 output["truth"].cpu().numpy()])
        
        gnn_recarray = np.rec.fromarrays(gnn_results, names=["senders", "receivers", "score", "truth"])

        with open(os.path.join(output_dir, os.path.split(batch.event_file[0])[-1][-4:] + ".npz"), 'wb') as f:
            np.save(f, gnn_recarray)